In [ ]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder

!pip install bayesian-optimization

from sklearn.model_selection import KFold,StratifiedKFold   # K-fold CV 
from bayes_opt import BayesianOptimization  # 베이지안 최적화 라이브러리
from functools import partial               # 함수 변수 고정
import lightgbm as lgb                      # LightGBM 라이브러리
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_squared_log_error

import warnings
warnings.filterwarnings(action = 'ignore')

from imblearn.over_sampling import SMOTE

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


  Created wheel for bayesian-optimization: filename=bayesian_optimization-1.2.0-cp36-none-any.whl size=11685 sha256=8e2d94b7848ac8734d39a90d527292b6e8136750364238732ad7091d50a9d4f2
  Stored in directory: /root/.cache/pip/wheels/5a/56/ae/e0e3c1fc1954dc3ec712e2df547235ed072b448094d8f94aec
Successfully built bayesian-optimization


In [ ]:
data=pd.read_csv('/content/drive/My Drive/Colab Notebooks/data/데이콘/제주/201901-202003.csv')
sub = pd.read_csv('/content/drive/My Drive/Colab Notebooks/data/데이콘/제주/submission.csv')

# Base Line 1
- Dacon Base Line + LightGBM 활용

In [ ]:
def grap_year(data):
    data = str(data)
    return int(data[:4])

def grap_month(data):
    data = str(data)
    return int(data[4:])

In [ ]:
# 날짜 처리
data = data.fillna('')
data['year'] = data['REG_YYMM'].apply(lambda x: grap_year(x))
data['month'] = data['REG_YYMM'].apply(lambda x: grap_month(x))
data = data.drop(['REG_YYMM'], axis=1)

In [ ]:
# 데이터 정제
df = data.copy()
df = df.drop(['CARD_CCG_NM', 'HOM_CCG_NM'], axis=1)

columns = ['CARD_SIDO_NM', 'STD_CLSS_NM', 'HOM_SIDO_NM', 'AGE', 'SEX_CTGO_CD', 'FLC', 'year', 'month']
df = df.groupby(columns).sum().reset_index(drop=False)

In [ ]:
df

In [ ]:
# 인코딩
dtypes = df.dtypes
encoders = {}
for column in df.columns:
    if str(dtypes[column]) == 'object':
        encoder = LabelEncoder()
        encoder.fit(df[column])
        encoders[column] = encoder
        
df_num = df.copy()        
for column in encoders.keys():
    encoder = encoders[column]
    df_num[column] = encoder.transform(df[column])

In [ ]:
df_num

,CARD_SIDO_NM,STD_CLSS_NM,HOM_SIDO_NM,AGE,SEX_CTGO_CD,FLC,year,month,CSTMR_CNT,AMT,CNT
0,0,0,0,1,1,1,2019,1,4,311200,4
1,0,0,0,1,1,1,2019,2,3,605000,3
2,0,0,0,1,1,1,2019,6,3,139000,3
3,0,0,0,1,1,1,2019,8,3,27500,3
4,0,0,0,1,1,1,2019,9,3,395500,3
...,...,...,...,...,...,...,...,...,...,...,...
1057389,16,40,16,6,2,5,2019,3,3,148000,4
1057390,16,40,16,6,2,5,2019,5,5,329800,7
1057391,16,40,16,6,2,5,2019,10,7,557800,7
1057392,16,40,16,6,2,5,2019,12,3,247800,3


In [ ]:
# feature, target 설정
train_num = df_num.sample(frac=1, random_state=0)
train_features = train_num.drop(['CSTMR_CNT', 'AMT', 'CNT'], axis=1)
train_target = np.log1p(train_num['AMT'])

In [ ]:
def lgb_cv(num_leaves, learning_rate, n_estimators, subsample, colsample_bytree, reg_alpha,reg_lambda, x_data=None, y_data=None, n_splits=5, output='model'):
    score = 0
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=201)
    models = []
    for train_index, valid_index in skf.split(x_data,x_data['month']):
       #print(train_index)
       x_train, y_train = x_data.iloc[train_index], y_data.iloc[train_index]
       x_valid, y_valid = x_data.iloc[valid_index], y_data.iloc[valid_index]
        
       model = lgb.LGBMRegressor(
               
           num_leaves = int(num_leaves), 
           learning_rate = learning_rate, 
           n_estimators = int(n_estimators), 
           subsample = np.clip(subsample, 0, 1), 
           colsample_bytree = np.clip(colsample_bytree, 0, 1), 
           reg_alpha = reg_alpha, 
           reg_lambda = reg_lambda,
           )
       
       model.fit(x_train, y_train)
       models.append(model)
        
       pred = model.predict(x_valid)
       true = y_valid
       RMSE = (mean_squared_error(true, pred)**0.5)*(-1)
       score += RMSE/n_splits
    
    if output == 'score':
        return score
    if output == 'model':
        return models

In [ ]:
func_fixed = partial(lgb_cv, x_data=train_features, y_data=train_target, n_splits=5, output='score') 

lgbBO = BayesianOptimization(
    func_fixed, 
    {
        'num_leaves': (4, 16),
        'learning_rate': (0.0001, 0.1),
        'n_estimators': (4, 16),
        'subsample': (0.01, 1),
        'colsample_bytree': (0.01, 1),
        'reg_alpha': (0.01, 10), 
        'reg_lambda': (0.01, 50),
    }, 
    random_state=4321
)
lgbBO.maximize(init_points=15, n_iter=5) #30

|   iter    |  target   | colsam... | learni... | n_esti... | num_le... | reg_alpha | reg_la... | subsample |
-------------------------------------------------------------------------------------------------------------
|  1        | -2.484    |  0.08009  |  0.08152  |  13.21    |  7.436    |  1.939    |  48.95    |  0.4122   |
|  2        | -2.498    |  0.7602   |  0.009006 |  7.719    |  11.43    |  4.605    |  10.92    |  0.6669   |
|  3        | -2.421    |  0.6819   |  0.09504  |  7.375    |  11.44    |  3.839    |  20.02    |  0.9432   |
|  4        | -2.416    |  0.9306   |  0.09484  |  8.506    |  8.108    |  6.651    |  2.126    |  0.2399   |
|  5        | -2.499    |  0.4358   |  0.007886 |  13.2     |  14.25    |  1.513    |  5.066    |  0.2782   |
|  6        | -2.482    |  0.03986  |  0.0838   |  11.17    |  15.17    |  4.96     |  18.83    |  0.2513   |
|  7        | -2.476    |  0.6058   |  0.03304  |  9.54     |  12.7     |  8.12     |  49.3     |  0.2347   |
|  8      

In [ ]:
params = lgbBO.max['params']
models = lgb_cv(
    params['num_leaves'],   
    params['learning_rate'], 
    params['n_estimators'], 
    params['subsample'], 
    params['colsample_bytree'], 
    params['reg_alpha'], 
    params['reg_lambda'], 
    x_data=train_features, y_data=train_target, n_splits=5, output='model')

In [ ]:
# 예측 템플릿 만들기
CARD_SIDO_NMs = df_num['CARD_SIDO_NM'].unique()
STD_CLSS_NMs  = df_num['STD_CLSS_NM'].unique()
HOM_SIDO_NMs  = df_num['HOM_SIDO_NM'].unique()
AGEs          = df_num['AGE'].unique()
SEX_CTGO_CDs  = df_num['SEX_CTGO_CD'].unique()
FLCs          = df_num['FLC'].unique()
years         = [2020]
months        = [4, 7]

temp = []
for CARD_SIDO_NM in CARD_SIDO_NMs:
    for STD_CLSS_NM in STD_CLSS_NMs:
        for HOM_SIDO_NM in HOM_SIDO_NMs:
            for AGE in AGEs:
                for SEX_CTGO_CD in SEX_CTGO_CDs:
                    for FLC in FLCs:
                        for year in years:
                            for month in months:
                                temp.append([CARD_SIDO_NM, STD_CLSS_NM, HOM_SIDO_NM, AGE, SEX_CTGO_CD, FLC, year, month])
temp = np.array(temp)
temp = pd.DataFrame(data=temp, columns=train_features.columns)

In [ ]:
preds = []
for model in models:
    pred = model.predict(temp)
    preds.append(pred)

pred = np.mean(preds, axis=0)

In [ ]:
temp['AMT'] = pred

In [ ]:
temp['AMT'] = np.round(pred, 0)
temp['REG_YYMM'] = temp['year']*100 + temp['month']
temp = temp[['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM', 'AMT']]
temp = temp.groupby(['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM']).sum().reset_index(drop=False)

In [ ]:
# 디코딩 
temp['CARD_SIDO_NM'] = encoders['CARD_SIDO_NM'].inverse_transform(temp['CARD_SIDO_NM'])
temp['STD_CLSS_NM'] = encoders['STD_CLSS_NM'].inverse_transform(temp['STD_CLSS_NM'])

In [ ]:
sub = sub.drop(['AMT'], axis=1)
sub = sub.merge(temp, left_on=['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM'], right_on=['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM'], how='left')
sub.index.name = 'id'
sub.to_csv('/content/drive/My Drive/Colab Notebooks/data/데이콘/제주/submission_baseline_1(6).csv', encoding='utf-8-sig')
sub.head()

,id,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT
id,,,,,
0,0,202004,강원,건강보조식품 소매업,16660.0
1,1,202004,강원,골프장 운영업,16660.0
2,2,202004,강원,과실 및 채소 소매업,16660.0
3,3,202004,강원,관광 민예품 및 선물용품 소매업,16660.0
4,4,202004,강원,그외 기타 분류안된 오락관련 서비스업,16660.0


오버피팅이 심하게 되는 것으로 보임. 독립변수들을 원핫인코딩으로 바꿔주는 것이 좋을 것 같음.

# Base Line 2

In [ ]:
data.head()

,REG_YYMM,CARD_SIDO_NM,CARD_CCG_NM,STD_CLSS_NM,HOM_SIDO_NM,HOM_CCG_NM,AGE,SEX_CTGO_CD,FLC,CSTMR_CNT,AMT,CNT
0,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,20s,1,1,4,311200,4
1,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,30s,1,2,7,1374500,8
2,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,30s,2,2,6,818700,6
3,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,40s,1,3,4,1717000,5
4,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,40s,1,4,3,1047300,3


In [ ]:
data.drop(['CARD_CCG_NM','HOM_CCG_NM','CSTMR_CNT','CNT'],axis=1,inplace=True)

In [ ]:
data=data.groupby(['REG_YYMM','CARD_SIDO_NM','STD_CLSS_NM','HOM_SIDO_NM','AGE','SEX_CTGO_CD','FLC'])['AMT'].sum().reset_index()

In [ ]:
data.head()

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,HOM_SIDO_NM,AGE,SEX_CTGO_CD,FLC,AMT
0,201901,강원,건강보조식품 소매업,강원,20s,1,1,311200
1,201901,강원,건강보조식품 소매업,강원,30s,1,2,5169295
2,201901,강원,건강보조식품 소매업,강원,30s,2,2,8618550
3,201901,강원,건강보조식품 소매업,강원,40s,1,2,454420
4,201901,강원,건강보조식품 소매업,강원,40s,1,3,6441165


In [ ]:
data.shape

(1057394, 8)

In [ ]:
# 이전 월 AMT 다음 월에 삽입하기위해 데이터 생성
sum_df=data.groupby(['REG_YYMM','CARD_SIDO_NM','STD_CLSS_NM'])['AMT'].sum().reset_index(name='sum_AMT')

# 다음 달 월에 삽일을 위해 연도 변수에 + 1
sum_df['REG_YYMM']= sum_df['REG_YYMM'].apply(lambda x: x +1 if x != 201912 else 202001)

# 19년 1월 데이터 삭제
data=data[data['REG_YYMM'] !=201901]

In [ ]:
# 전 월 AMT 삽입
data = data.merge(sum_df,left_on=['REG_YYMM','CARD_SIDO_NM','STD_CLSS_NM'],right_on=['REG_YYMM','CARD_SIDO_NM','STD_CLSS_NM'],how='left')
data.head()

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,HOM_SIDO_NM,AGE,SEX_CTGO_CD,FLC,AMT,sum_AMT
0,201902,강원,건강보조식품 소매업,강원,20s,1,1,605000,148354746.0
1,201902,강원,건강보조식품 소매업,강원,20s,2,1,912000,148354746.0
2,201902,강원,건강보조식품 소매업,강원,20s,2,2,216200,148354746.0
3,201902,강원,건강보조식품 소매업,강원,30s,1,2,8972970,148354746.0
4,201902,강원,건강보조식품 소매업,강원,30s,2,2,10469559,148354746.0


In [ ]:
data['year']=data['REG_YYMM'].apply(lambda x: int(str(x)[:4]))
data['month'] = data['REG_YYMM'].apply(lambda x: int(str(x)[4:]))
data.drop('REG_YYMM',axis=1,inplace=True)
data.head()

,CARD_SIDO_NM,STD_CLSS_NM,HOM_SIDO_NM,AGE,SEX_CTGO_CD,FLC,AMT,sum_AMT,year,month
0,강원,건강보조식품 소매업,강원,20s,1,1,605000,148354746.0,2019,2
1,강원,건강보조식품 소매업,강원,20s,2,1,912000,148354746.0,2019,2
2,강원,건강보조식품 소매업,강원,20s,2,2,216200,148354746.0,2019,2
3,강원,건강보조식품 소매업,강원,30s,1,2,8972970,148354746.0,2019,2
4,강원,건강보조식품 소매업,강원,30s,2,2,10469559,148354746.0,2019,2


In [ ]:
sum_df['year']=sum_df['REG_YYMM'].apply(lambda x: int(str(x)[:4]))
sum_df['month'] = sum_df['REG_YYMM'].apply(lambda x: int(str(x)[4:]))
sum_df.drop('REG_YYMM',axis=1,inplace=True)
sum_df.head()

,CARD_SIDO_NM,STD_CLSS_NM,sum_AMT,year,month
0,강원,건강보조식품 소매업,148354746,2019,2
1,강원,골프장 운영업,1229200312,2019,2
2,강원,과실 및 채소 소매업,1286979106,2019,2
3,강원,관광 민예품 및 선물용품 소매업,30656770,2019,2
4,강원,그외 기타 스포츠시설 운영업,13101500,2019,2


In [ ]:
data.loc[data['year'] ==2019,'corona'] = 0
data.loc[data['year'] ==2020,'corona'] = 1

In [ ]:
data.loc[(data['month']>=3) & (data['month']<=5),'season'] = 'spring'
data.loc[(data['month']>=6) & (data['month']<8),'season'] = 'summer'
data.loc[(data['month']>=9) & (data['month']<=11),'season'] = 'fall'
data.loc[(data['month']==12) | (data['month']<=2),'season'] = 'winter'

In [ ]:
data.dtypes

CARD_SIDO_NM     object
STD_CLSS_NM      object
HOM_SIDO_NM      object
AGE              object
SEX_CTGO_CD       int64
FLC               int64
AMT               int64
sum_AMT         float64
year              int64
month             int64
corona          float64
season           object
dtype: object

In [ ]:
for col in data.columns:
  if col !='AMT' and col !='sum_AMT':
    data[col] = data[col].astype('category')
data.dtypes

CARD_SIDO_NM    category
STD_CLSS_NM     category
HOM_SIDO_NM     category
AGE             category
SEX_CTGO_CD     category
FLC             category
AMT                int64
sum_AMT          float64
year            category
month           category
corona          category
season          category
dtype: object

In [ ]:
x_data = data.drop('AMT',axis=1)
y_data = np.log1p(data['AMT'])

In [ ]:
# data.to_csv('/content/drive/My Drive/Colab Notebooks/data/데이콘/제주/train_df.csv', encoding='utf-8-sig',index=False)

In [ ]:
# test 템플릿 만드기
CARD_SIDO_NMs = data['CARD_SIDO_NM'].unique()
STD_CLSS_NMs  = data['STD_CLSS_NM'].unique()
HOM_SIDO_NMs  = data['HOM_SIDO_NM'].unique()
AGEs          = data['AGE'].unique()
SEX_CTGO_CDs  = data['SEX_CTGO_CD'].unique()
FLCs          = data['FLC'].unique()
years           = [2020]
months           = [4,7]
coronas = [1]
#seasons         = ['spring','summer']

temp = []
for CARD_SIDO_NM in CARD_SIDO_NMs:
    for STD_CLSS_NM in STD_CLSS_NMs:
        for HOM_SIDO_NM in HOM_SIDO_NMs:
            for AGE in AGEs:
                for SEX_CTGO_CD in SEX_CTGO_CDs:
                    for FLC in FLCs:
                      for year in years:
                        for month in months:
                          for corona in coronas:
                            if month ==4:
                              season ='spring'
                            else:
                              season ='summer'                         
                          
                            temp.append([CARD_SIDO_NM, STD_CLSS_NM, HOM_SIDO_NM, AGE, SEX_CTGO_CD, FLC,year,month,corona,season])

temp = np.array(temp)
temp = pd.DataFrame(data=temp, columns=['CARD_SIDO_NM', 'STD_CLSS_NM', 'HOM_SIDO_NM', 'AGE', 'SEX_CTGO_CD', 'FLC','year','month','corona','season'])

In [ ]:
temp

,CARD_SIDO_NM,STD_CLSS_NM,HOM_SIDO_NM,AGE,SEX_CTGO_CD,FLC,year,month,corona,season
0,강원,건강보조식품 소매업,강원,20s,1,1,2020,4,1,spring
1,강원,건강보조식품 소매업,강원,20s,1,1,2020,7,1,summer
2,강원,건강보조식품 소매업,강원,20s,1,2,2020,4,1,spring
3,강원,건강보조식품 소매업,강원,20s,1,2,2020,7,1,summer
4,강원,건강보조식품 소매업,강원,20s,1,3,2020,4,1,spring
...,...,...,...,...,...,...,...,...,...,...
1658855,충북,정기 항공 운송업,전남,10s,2,3,2020,7,1,summer
1658856,충북,정기 항공 운송업,전남,10s,2,4,2020,4,1,spring
1658857,충북,정기 항공 운송업,전남,10s,2,4,2020,7,1,summer
1658858,충북,정기 항공 운송업,전남,10s,2,5,2020,4,1,spring


In [ ]:
data

,CARD_SIDO_NM,STD_CLSS_NM,HOM_SIDO_NM,AGE,SEX_CTGO_CD,FLC,AMT,sum_AMT,year,month,corona,season
0,강원,건강보조식품 소매업,강원,20s,1,1,605000,148354746.0,2019,2,0.0,winter
1,강원,건강보조식품 소매업,강원,20s,2,1,912000,148354746.0,2019,2,0.0,winter
2,강원,건강보조식품 소매업,강원,20s,2,2,216200,148354746.0,2019,2,0.0,winter
3,강원,건강보조식품 소매업,강원,30s,1,2,8972970,148354746.0,2019,2,0.0,winter
4,강원,건강보조식품 소매업,강원,30s,2,2,10469559,148354746.0,2019,2,0.0,winter
...,...,...,...,...,...,...,...,...,...,...,...,...
986443,충북,휴양콘도 운영업,충북,40s,1,4,385700,47085830.0,2020,3,1.0,spring
986444,충북,휴양콘도 운영업,충북,50s,1,4,790350,47085830.0,2020,3,1.0,spring
986445,충북,휴양콘도 운영업,충북,50s,2,4,99000,47085830.0,2020,3,1.0,spring
986446,충북,휴양콘도 운영업,충북,60s,1,5,228800,47085830.0,2020,3,1.0,spring


In [ ]:
# sum_df.to_csv('/content/drive/My Drive/Colab Notebooks/data/데이콘/제주/sum_df.csv', encoding='utf-8-sig',index=False)
# temp.to_csv('/content/drive/My Drive/Colab Notebooks/data/데이콘/제주/temp.csv', encoding='utf-8-sig',index=False)

In [ ]:
def lgb_cv(max_depth,num_leaves, learning_rate, n_estimators, subsample, colsample_bytree, reg_alpha,reg_lambda, x_data=None, y_data=None, n_splits=5, output='model'):
    score = 0
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=201)
    models = []
    for train_index, valid_index in skf.split(x_data,x_data['STD_CLSS_NM']):
       #print(train_index)
       x_train, y_train = x_data.iloc[train_index], y_data.iloc[train_index]
       x_valid, y_valid = x_data.iloc[valid_index], y_data.iloc[valid_index]
        
       model = lgb.LGBMRegressor(
               
           num_leaves = int(num_leaves), 
           learning_rate = learning_rate, 
           n_estimators = int(n_estimators), 
           subsample = np.clip(subsample, 0, 1), 
           colsample_bytree = np.clip(colsample_bytree, 0, 1), 
           reg_alpha = reg_alpha, 
           reg_lambda = reg_lambda,
           )
       
       model.fit(x_train, y_train)
       models.append(model)
        
       pred = model.predict(x_valid)
       true = y_valid
       RMSLE = (mean_squared_log_error(true, pred))*(-1)
       score += RMSLE/n_splits
    
    if output == 'score':
        return score
    if output == 'model':
        return models

In [ ]:
func_fixed = partial(lgb_cv, x_data=x_data, y_data=y_data, n_splits=5, output='score') 

lgbBO = BayesianOptimization(
    func_fixed, 
    {
        'max_depth' : (16,1024),
        'num_leaves': (16, 1024),
        'learning_rate': (0.0001, 0.1),
        'n_estimators': (16, 1024),
        'subsample': (0.01, 1),
        'colsample_bytree': (0.01, 1),
        'reg_alpha': (0.01, 10), 
        'reg_lambda': (0.01, 50),
    }, 
    random_state=4321
)
lgbBO.maximize(init_points=25, n_iter=5) #30  

|   iter    |  target   | colsam... | learni... | max_depth | n_esti... | num_le... | reg_alpha | reg_la... | subsample |
-------------------------------------------------------------------------------------------------------------------------
|  1        | -0.02273  |  0.08009  |  0.08152  |  790.0    |  304.6    |  210.6    |  9.789    |  20.32    |  0.7602   |
|  2        | -0.02288  |  0.09826  |  0.03106  |  639.9    |  479.6    |  236.1    |  6.639    |  33.94    |  0.9508   |
|  3        | -0.02149  |  0.2884   |  0.06202  |  402.4    |  419.6    |  966.2    |  9.3      |  47.42    |  0.3817   |
|  4        | -0.01562  |  0.3489   |  0.06651  |  58.66    |  250.1    |  449.5    |  0.7886   |  38.33    |  0.8554   |
|  5        | -0.02604  |  0.1589   |  0.0102   |  289.1    |  46.4     |  860.5    |  5.98     |  46.56    |  0.5006   |
|  6        | -0.01739  |  0.3827   |  0.02445  |  622.6    |  348.3    |  481.4    |  7.252    |  40.59    |  0.9862   |
|  7        | -0.02113  

In [ ]:
params = lgbBO.max['params']
models = lgb_cv(
    params['max_depth'],
    params['num_leaves'],   
    params['learning_rate'], 
    params['n_estimators'], 
    params['subsample'], 
    params['colsample_bytree'], 
    params['reg_alpha'], 
    params['reg_lambda'], 
    x_data=x_data, y_data=y_data, n_splits=5, output='model')

In [ ]:
for col in temp.columns:
  temp[col] = temp[col].astype('category')

In [ ]:
preds = []
for model in models:
    pred = model.predict(temp)
    preds.append(pred)

pred = np.mean(preds, axis=0)

temp['AMT'] = pred

In [ ]:
temp['REG_YYMM']= temp['year'].astype('object') +temp['month'].astype('object')
temp['REG_YYMM'] = temp['REG_YYMM'].astype('int64')

In [ ]:
temp = temp[['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM', 'AMT']]
temp['AMT'] = np.expm1(temp['AMT'])
temp = temp.groupby(['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM']).sum().reset_index(drop=False)

In [ ]:
temp = temp.reset_index()
temp['AMT'] = np.round(temp['AMT'], 0)
temp['CARD_SIDO_NM'] = temp['CARD_SIDO_NM'].astype('object')
temp['STD_CLSS_NM'] = temp['STD_CLSS_NM'].astype('object')
temp['AMT'] = temp['AMT'].astype('int64')
# temp.to_csv('/content/drive/My Drive/Colab Notebooks/data/데이콘/제주/submission_baseline_3(7).csv', encoding='utf-8-sig',index=False)
# temp.head()

In [ ]:
sub = pd.read_csv('/content/drive/My Drive/Colab Notebooks/data/데이콘/제주/submission.csv')
# sub = sub.drop(['AMT'], axis=1)
# sub.merge(temp,on=['REG_YYMM','CARD_SIDO_NM','STD_CLSS_NM'])

In [ ]:
sub['AMT'] = temp['AMT']
sub.head()

,id,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT
0,0,202004,강원,건강보조식품 소매업,13886229
1,1,202004,강원,골프장 운영업,385097701
2,2,202004,강원,과실 및 채소 소매업,165273097
3,3,202004,강원,관광 민예품 및 선물용품 소매업,10660922
4,4,202004,강원,그외 기타 분류안된 오락관련 서비스업,5822870


In [ ]:
sub.to_csv('/content/drive/My Drive/Colab Notebooks/data/데이콘/제주/submission_baseline_2(13).csv', encoding='utf-8-sig',index=False)

# Base Line 3
- Smote 적용
 - 이를 위해 라벨 인코딩을 해야함.

## train 데이터 만들기

In [ ]:
data.head()

In [ ]:
data.drop(['CARD_CCG_NM','HOM_CCG_NM','CSTMR_CNT','CNT'],axis=1,inplace=True)
data=data.groupby(['REG_YYMM','CARD_SIDO_NM','STD_CLSS_NM','HOM_SIDO_NM','AGE','SEX_CTGO_CD','FLC'])['AMT'].sum().reset_index()

In [ ]:
# for col in data.drop('AMT',axis=1):
#   data[col] = data[col].astype('str')

In [ ]:
data['year']=data['REG_YYMM'].apply(lambda x: int(str(x)[:4]))
data['month'] = data['REG_YYMM'].apply(lambda x: int(str(x)[4:]))

In [ ]:
data.loc[data['year'] ==2019,'corona'] = 0
data.loc[data['year'] ==2020,'corona'] = 1

In [ ]:
data.loc[(data['month']>=3) & (data['month']<=5),'season'] = 'spring'
data.loc[(data['month']>=6) & (data['month']<=8),'season'] = 'summer'
data.loc[(data['month']>=9) & (data['month']<=11),'season'] = 'fall'
data.loc[(data['month']==12) | (data['month']<=2),'season'] = 'winter'

In [ ]:
data['next_month']=data['REG_YYMM'].apply(lambda x: x+1 if x != 201912 else 202001)
data.head()

In [ ]:
# 이전 월마다 업종의  AMT 구하기
nm_sum = data.groupby(['next_month','CARD_SIDO_NM','STD_CLSS_NM','HOM_SIDO_NM'])['AMT'].sum().reset_index(name='previous_AMT')
nm_sum.head()

In [ ]:
train_df=\
data[data['REG_YYMM'] !=201901].drop('next_month',axis=1).merge(nm_sum,how='left',
                                                                left_on=['REG_YYMM','CARD_SIDO_NM','STD_CLSS_NM','HOM_SIDO_NM'],
                                                                right_on=['next_month','CARD_SIDO_NM','STD_CLSS_NM','HOM_SIDO_NM'])
train_df.drop('next_month',axis=1,inplace=True)
train_df['previous_AMT'] = train_df['previous_AMT'].fillna(0)
train_df.head()

In [ ]:
data.head()

## test 데이터 만들기

In [ ]:
# 기본 test 템플릿 만들기
REG_YYMMs = [202004,202007]
CARD_SIDO_NMs = data['CARD_SIDO_NM'].unique()
STD_CLSS_NMs  = data['STD_CLSS_NM'].unique()
HOM_SIDO_NMs  = data['HOM_SIDO_NM'].unique()
AGEs          = data['AGE'].unique()
SEX_CTGO_CDs  = data['SEX_CTGO_CD'].unique()
FLCs          = data['FLC'].unique()

temp = []
for REG_YYMM in REG_YYMMs:
  for CARD_SIDO_NM in CARD_SIDO_NMs:
      for STD_CLSS_NM in STD_CLSS_NMs:
          for HOM_SIDO_NM in HOM_SIDO_NMs:
              for AGE in AGEs:
                  for SEX_CTGO_CD in SEX_CTGO_CDs:
                      for FLC in FLCs:
                        temp.append([REG_YYMM,CARD_SIDO_NM, STD_CLSS_NM, HOM_SIDO_NM, AGE, SEX_CTGO_CD, FLC])

temp = np.array(temp)
temp = pd.DataFrame(data=temp, columns=['REG_YYMM','CARD_SIDO_NM', 'STD_CLSS_NM', 'HOM_SIDO_NM', 'AGE', 'SEX_CTGO_CD', 'FLC'])

In [ ]:
temp['year']=temp['REG_YYMM'].apply(lambda x: int(str(x)[:4]))
temp['month'] = temp['REG_YYMM'].apply(lambda x: int(str(x)[4:]))

temp['corona']=1

temp.loc[(temp['month']>=3) & (temp['month']<=5),'season'] = 'spring'
temp.loc[(temp['month']>=6) & (temp['month']<8),'season'] = 'summer'
temp.loc[(temp['month']>=9) & (temp['month']<=11),'season'] = 'fall'
temp.loc[(temp['month']==12) | (temp['month']<=2),'season'] = 'winter'

#temp['next_month'] = temp['REG_YYMM'].apply(lambda x: int(x)+1 if int(x) != 201912 else 202001)
#temp['next_month'] = temp['next_month'].astype('int64')
temp['REG_YYMM'] = temp['REG_YYMM'].astype('int64')
temp.head()

In [ ]:
test_df=temp.merge(nm_sum,how='left',
                   left_on=['REG_YYMM','CARD_SIDO_NM','STD_CLSS_NM','HOM_SIDO_NM'],
                   right_on=['next_month','CARD_SIDO_NM','STD_CLSS_NM','HOM_SIDO_NM'])

test_df.drop('next_month',axis=1,inplace=True)
test_df['previous_AMT']=test_df['previous_AMT'].fillna(0)
test_df.head()

In [ ]:
train_df.drop('REG_YYMM',axis=1,inplace=True)
test_df.drop('REG_YYMM',axis=1,inplace=True)

In [ ]:
# train, test type 맞춰주기
for col in test_df.columns:
  test_df[col] = test_df[col].astype(train_df[col].dtype)

In [ ]:
train_df

In [ ]:
def make_encoders(df):
  encoders={}
  for col in df.drop(['AMT','previous_AMT','corona'],axis=1).columns:
    con_list =[]
    encoder={}
    for idx, content in enumerate(df[col].unique()):
      encoder[content]=idx
    
    encoders[col] = encoder
  
  return encoders

In [ ]:
kk=make_encoders(train_df)

In [ ]:
for col in kk.keys():
  train_df[col] = train_df[col].apply(lambda x: kk[col][x])
  test_df[col] = test_df[col].apply(lambda x: kk[col][x])

In [ ]:
train_df

In [ ]:
train_df['month'].map(my_dict)

In [ ]:
test_df

---

## 모델링

In [ ]:
for col in train_df.columns:
  if col !='AMT' and col !='previous_AMT':
    train_df[col] = train_df[col].astype('category')
    test_df[col] = test_df[col].astype('category')

In [ ]:
x_data = train_df.drop('AMT',axis=1)
y_data = np.log1p(train_df['AMT'])

In [ ]:
def lgb_cv(max_depth,num_leaves, learning_rate, n_estimators, subsample, colsample_bytree, reg_alpha,reg_lambda, x_data=None, y_data=None, n_splits=5, output='model'):
    score = 0
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=201)
    models = []
    for train_index, valid_index in skf.split(x_data,x_data['STD_CLSS_NM']):
       x_train, y_train = x_data.iloc[train_index], y_data.iloc[train_index]
       x_valid, y_valid = x_data.iloc[valid_index], y_data.iloc[valid_index]

       # Smote를 위해 데이터 병합
       x_train['AMT'] = y_train

       # Smote
       sm = SMOTE(ratio='auto', kind='regular')
       data_over,_ = sm.fit_sample(x_train,x_train['STD_CLSS_NM'])
       X_resampled = data_over[:,:-1]
       y_resampled = data_over[:,-1]

       x_valid, y_valid = x_data.iloc[valid_index], y_data.iloc[valid_index]

       model = lgb.LGBMRegressor(
               
           num_leaves = int(num_leaves), 
           learning_rate = learning_rate, 
           n_estimators = int(n_estimators), 
           subsample = np.clip(subsample, 0, 1), 
           colsample_bytree = np.clip(colsample_bytree, 0, 1), 
           reg_alpha = reg_alpha, 
           reg_lambda = reg_lambda,
           )
       
       model.fit(X_resampled, y_resampled)
       models.append(model)
        
       pred = model.predict(x_valid)
       true = y_valid
       RMSLE = (mean_squared_log_error(true, pred))*(-1)
       score += RMSLE/n_splits
    
    if output == 'score':
        return score
    if output == 'model':
        return models

In [ ]:
func_fixed = partial(lgb_cv, x_data=x_data, y_data=y_data, n_splits=5, output='score') 

lgbBO = BayesianOptimization(
    func_fixed, 
    {
        'max_depth' : (16,1024),
        'num_leaves': (16, 1024),
        'learning_rate': (0.0001, 0.1),
        'n_estimators': (16, 1024),
        'subsample': (0.01, 1),
        'colsample_bytree': (0.01, 1),
        'reg_alpha': (0.01, 10), 
        'reg_lambda': (0.01, 50),
    }, 
    random_state=4321
)
lgbBO.maximize(init_points=25, n_iter=5) #30  

In [ ]:
params = lgbBO.max['params']
models = lgb_cv(
    params['max_depth'],
    params['num_leaves'],   
    params['learning_rate'], 
    params['n_estimators'], 
    params['subsample'], 
    params['colsample_bytree'], 
    params['reg_alpha'], 
    params['reg_lambda'], 
    x_data=x_data, y_data=y_data, n_splits=5, output='model')

In [ ]:
preds = []
for model in models:
    pred = model.predict(test_df)
    preds.append(pred)

pred = np.mean(preds, axis=0)

test_df['AMT'] = pred
test_df['AMT'] = np.expm1(test_df['AMT'])

In [ ]:
test_df.head()

In [ ]:
def make_decoding(df,col):
  if col in kk.keys():
    tt = [name for x in df[col] for name,num in kk[col].items() if x == num]
  return tt

In [ ]:
for c in test_df.columns:
 test_df[c] = make_decoding(test_df,c)

In [ ]:
sub['AMT'] = test_df.groupby(['year','month','CARD_SIDO_NM','STD_CLSS_NM'])['AMT'].sum().reset_index()['AMT']

In [ ]:
sub['AMT'] = sub['AMT'].astype('int64')

In [ ]:
sub.to_csv('/content/drive/My Drive/Colab Notebooks/data/데이콘/제주/sub(0702).csv', encoding='utf-8-sig',index=False)

# BaseLine 4
- previous_AMT, CNT,CSTMR_CNT 변수를 만든 데이터프레임

In [ ]:
df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/data/데이콘/제주/train_df_noNAN.csv')
df_train=pd.read_csv('/content/drive/My Drive/Colab Notebooks/data/데이콘/제주/df_train.csv')
df_test=pd.read_csv('/content/drive/My Drive/Colab Notebooks/data/데이콘/제주/df_test.csv')
sub = pd.read_csv('/content/drive/My Drive/Colab Notebooks/data/데이콘/제주/submission.csv')

df_train에 AMT 값이 없기 때문에 값을 넣자.

In [ ]:
df_train = df_train.merge(df.drop(['CSTMR_CNT','CNT'],axis=1),
                          on=['REG_YYMM','CARD_SIDO_NM','CARD_CCG_NM','STD_CLSS_NM','HOM_SIDO_NM','HOM_CCG_NM','AGE','SEX_CTGO_CD','FLC'],
                          how='left')

df_train과 df_test 모두 na값을 0으로 바꿔주자.

In [ ]:
df_train.fillna(0,inplace=True)
df_test.fillna(0,inplace=True)

In [ ]:
#df_train = df_train[df_train['AMT'] !=0]

In [ ]:
def make_encoders(df):
  encoders={}
  for col in ['REG_YYMM', 'CARD_SIDO_NM', 'CARD_CCG_NM', 'STD_CLSS_NM', 'HOM_SIDO_NM','HOM_CCG_NM', 'AGE', 'SEX_CTGO_CD', 'FLC']:
    con_list =[]
    encoder={}
    for idx, content in enumerate(df[col].unique()):
      encoder[content]=idx

    if col == 'REG_YYMM':
        encoder[202004] = idx+1
        encoder[202007] = idx+2
    
    encoders[col] = encoder
  
  return encoders

In [ ]:
encoders = make_encoders(df_train)

In [ ]:
for col in encoders.keys():
  df_train[col] = df_train[col].map(encoders[col])
  df_test[col] = df_test[col].map(encoders[col])

In [ ]:
# decoder 만들기

decoders={}
for kk,vv in encoders.items():
  decoder={}
  for k,v in vv.items():
    decoder[v]=k
  decoders[kk] = decoder

In [ ]:
for col in ['REG_YYMM', 'CARD_SIDO_NM', 'CARD_CCG_NM', 'STD_CLSS_NM', 'HOM_SIDO_NM','HOM_CCG_NM', 'AGE', 'SEX_CTGO_CD', 'FLC']:
  df_train[col] = df_train[col].astype('category')
  df_test[col] = df_test[col].astype('category')

In [ ]:
x_data = df_train.drop('AMT',axis=1)
y_data = np.log1p(df_train['AMT'])

In [ ]:
def lgb_cv(max_depth,num_leaves, learning_rate, n_estimators, subsample, colsample_bytree, reg_alpha,reg_lambda, x_data=None, y_data=None, n_splits=5, output='model'):
    score = 0
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=201)
    models = []
    for train_index, valid_index in skf.split(x_data,x_data['STD_CLSS_NM']):
       #print(train_index)
       x_train, y_train = x_data.iloc[train_index], y_data.iloc[train_index]
       x_valid, y_valid = x_data.iloc[valid_index], y_data.iloc[valid_index]
        
       model = lgb.LGBMRegressor(
               
           num_leaves = int(num_leaves), 
           learning_rate = learning_rate, 
           n_estimators = int(n_estimators), 
           subsample = np.clip(subsample, 0, 1), 
           colsample_bytree = np.clip(colsample_bytree, 0, 1), 
           reg_alpha = reg_alpha, 
           reg_lambda = reg_lambda,
           )
       
       model.fit(x_train, y_train)
       models.append(model)
        
       pred = model.predict(x_valid)
       true = y_valid
       RMSLE = (mean_squared_log_error(true, pred))*(-1)
       score += RMSLE/n_splits
    
    if output == 'score':
        return score
    if output == 'model':
        return models

In [ ]:
func_fixed = partial(lgb_cv, x_data=x_data, y_data=y_data, n_splits=5, output='score') 

lgbBO = BayesianOptimization(
    func_fixed, 
    {
        'max_depth' : (16,256),
        'num_leaves': (16, 256),
        'learning_rate': (0.0001, 0.1),
        'n_estimators': (5, 10),
        'subsample': (0.01, 1),
        'colsample_bytree': (0.01, 1),
        'reg_alpha': (0.01, 10), 
        'reg_lambda': (0.01, 50),
    }, 
    random_state=4321
)
lgbBO.maximize(init_points=5, n_iter=5) #30  

|   iter    |  target   | colsam... | learni... | max_depth | n_esti... | num_le... | reg_alpha | reg_la... | subsample |
-------------------------------------------------------------------------------------------------------------------------
|  1        | -2.174    |  0.08009  |  0.08152  |  200.3    |  6.432    |  62.34    |  9.789    |  20.32    |  0.7602   |
|  2        | -2.191    |  0.09826  |  0.03106  |  164.5    |  7.3      |  68.39    |  6.639    |  33.94    |  0.9508   |
|  3        | -2.098    |  0.2884   |  0.06202  |  108.0    |  7.002    |  242.2    |  9.3      |  47.42    |  0.3817   |
|  4        | -2.129    |  0.3489   |  0.06651  |  26.16    |  6.161    |  119.2    |  0.7886   |  38.33    |  0.8554   |
|  5        | -2.206    |  0.1589   |  0.0102   |  81.02    |  5.151    |  217.1    |  5.98     |  46.56    |  0.5006   |
|  6        | -2.221    |  1.0      |  0.0001   |  256.0    |  10.0     |  256.0    |  0.01     |  0.01     |  1.0      |
|  7        | -2.049    

In [ ]:
params = lgbBO.max['params']
models = lgb_cv(
    params['max_depth'],
    params['num_leaves'],   
    params['learning_rate'], 
    params['n_estimators'], 
    params['subsample'], 
    params['colsample_bytree'], 
    params['reg_alpha'], 
    params['reg_lambda'], 
    x_data=x_data, y_data=y_data, n_splits=5, output='model')

In [ ]:
preds = []
for model in models:
    pred = model.predict(df_test)
    preds.append(pred)

pred = np.mean(preds, axis=0)

df_test['AMT'] = pred
df_test['AMT'] = df_test['AMT'].fillna(0)
df_test['AMT'] = np.expm1(df_test['AMT'])

In [ ]:
df_test.isna().sum()

REG_YYMM                      0
CARD_SIDO_NM                  0
CARD_CCG_NM                   0
STD_CLSS_NM                   0
HOM_SIDO_NM                   0
HOM_CCG_NM                    0
AGE                           0
SEX_CTGO_CD                   0
FLC                           0
previous_CARD_AMT_3           0
previous_CARD_AMT_6           0
previous_CARD_AMT_9           0
previous_CARD_AMT_12          0
previous_HOM_AMT_3            0
previous_HOM_AMT_6            0
previous_HOM_AMT_9            0
previous_HOM_AMT_12           0
previous_ASF_AMT_3            0
previous_ASF_AMT_6            0
previous_ASF_AMT_9            0
previous_ASF_AMT_12           0
previous_CARD_CNT_3           0
previous_CARD_CNT_6           0
previous_CARD_CNT_9           0
previous_CARD_CNT_12          0
previous_HOM_CNT_3            0
previous_HOM_CNT_6            0
previous_HOM_CNT_9            0
previous_HOM_CNT_12           0
previous_ASF_CNT_3            0
previous_ASF_CNT_6            0
previous

In [ ]:
for col in decoders.keys():
  df_test[col] = df_test[col].map(decoders[col])

In [ ]:
#df_test = df_test.groupby(['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM'])['AMT'].sum().reset_index()
df_test['AMT'] = df_test['AMT'].fillna(0)
df_test['AMT'] = df_test['AMT'].astype('int')

In [ ]:
sub.drop('AMT',axis=1,inplace=True)
sub = sub.merge(df_test,on=['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM'],how='left')

In [ ]:
sub.to_csv('/content/drive/My Drive/Colab Notebooks/data/데이콘/제주/submission_baseline_2(15).csv', encoding='utf-8-sig',index=False)

In [ ]:
sub.head()

,id,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT
0,0,202004,강원,건강보조식품 소매업,60903
1,1,202004,강원,골프장 운영업,577833
2,2,202004,강원,과실 및 채소 소매업,457367
3,3,202004,강원,관광 민예품 및 선물용품 소매업,23971
4,4,202004,강원,그외 기타 분류안된 오락관련 서비스업,28
